In [36]:
from utils import get_data, create_custom_color_palette

# type_data = "all_data"
type_data = "same_seed"
# type_data = "different_seed"

data = get_data(type_data)

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [38]:
def process_action_types_by_model(all_data):
    voter_actions = {}
    candidate_actions = {}

    for sim_idx, sim_data in enumerate(all_data):
        for agent in sim_data["people_agents"]:
            model = agent["model_name"]

            if model:
                if model not in voter_actions:
                    voter_actions[model] = {"POST": 0, "REPLY": 0, "LIKE": 0}

                for day_actions in agent.get("all_posting_actions", []):
                    for action in day_actions.get("actions", []):
                        action_type = action.get("action")
                        if action_type in ["POST", "REPLY", "LIKE"]:
                            voter_actions[model][action_type] += 1

        for agent in sim_data["candidate_agents"]:
            model = agent["model_name"]

            if model:
                if model not in candidate_actions:
                    candidate_actions[model] = {"POST": 0, "REPLY": 0, "LIKE": 0}

                for day_actions in agent.get("all_posting_actions", []):
                    for action in day_actions.get("actions", []):
                        action_type = action.get("action")
                        if action_type in ["POST", "REPLY", "LIKE"]:
                            candidate_actions[model][action_type] += 1

    return voter_actions, candidate_actions

In [39]:
def create_single_chart_for_subplot(
    model_actions, person_type_name, ax, action_types, colors
):
    """Create a horizontal stacked bar chart on a given axis"""
    if not model_actions:
        ax.text(
            0.5,
            0.5,
            f"No data found for {person_type_name}",
            transform=ax.transAxes,
            ha="center",
            va="center",
        )
        return

    models = list(model_actions.keys())

    action_matrix = []
    for model in models:
        row = []
        for action_type in action_types:
            row.append(model_actions[model].get(action_type, 0))
        action_matrix.append(row)

    model_totals = [sum(row) for row in action_matrix]
    sorted_indices = sorted(
        range(len(models)), key=lambda i: model_totals[i], reverse=True
    )

    sorted_models = [models[i] for i in sorted_indices]
    sorted_matrix = [action_matrix[i] for i in sorted_indices]

    left = np.zeros(len(sorted_models))

    for i, action_type in enumerate(action_types):
        values = [row[i] for row in sorted_matrix]
        bars = ax.barh(
            range(len(sorted_models)),
            values,
            left=left,
            label=action_type,
            color=colors[i],
        )
        left += values

    ax.set_yticks(range(len(sorted_models)))
    ax.set_yticklabels([model.split("/")[-1] for model in sorted_models])
    ax.tick_params(labelsize=14)
    ax.set_xlabel("Action Count", fontsize=14)
    ax.set_title(f"{person_type_name} Action Types by Model", fontsize=15)
    ax.invert_yaxis()


def create_combined_horizontal_stacked_bar_chart(
    voter_actions, candidate_actions, type_data=None
):
    """Create side-by-side horizontal stacked bar charts for voters and candidates"""
    action_types = ["POST", "REPLY", "LIKE"]
    colors = create_custom_color_palette(len(action_types))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    create_single_chart_for_subplot(voter_actions, "Voters", ax1, action_types, colors)

    create_single_chart_for_subplot(
        candidate_actions, "Candidates", ax2, action_types, colors
    )

    title = "Action Types by Model"
    if type_data:
        title += f" ({type_data})"
    fig.suptitle(title, fontsize=16, fontweight="bold")

    handles, labels = ax1.get_legend_handles_labels()
    fig.legend(
        handles,
        labels,
        loc="upper center",
        bbox_to_anchor=(0.5, 0.05),
        ncol=len(action_types),
        fontsize=12,
    )

    plt.rcParams["figure.dpi"] = 600
    plt.rcParams["savefig.dpi"] = 600
    plt.tight_layout()
    plt.subplots_adjust(top=0.92, bottom=0.1)  # Make room for title and legend
    plt.show()


def print_action_totals(model_actions, person_type_name):
    """Print total message counts and breakdown by action type"""
    action_types = ["POST", "REPLY", "LIKE"]

    action_totals = {action_type: 0 for action_type in action_types}
    total_messages = 0

    for model, actions in model_actions.items():
        for action_type in action_types:
            count = actions.get(action_type, 0)
            action_totals[action_type] += count
            total_messages += count

    print(f"\n### {person_type_name} Action Summary ###")
    print(f"Total Messages: {total_messages:,}")
    print("Breakdown by Action Type:")
    for action_type in action_types:
        count = action_totals[action_type]
        percentage = (count / total_messages * 100) if total_messages > 0 else 0
        print(f"  {action_type}: {count:,} ({percentage:.1f}%)")

    return total_messages, action_totals


def plot_action_types_by_model(all_data, type_data=None):
    voter_actions, candidate_actions = process_action_types_by_model(all_data)

    voter_total, voter_action_totals = print_action_totals(voter_actions, "Voters")
    candidate_total, candidate_action_totals = print_action_totals(
        candidate_actions, "Candidates"
    )

    grand_total = voter_total + candidate_total
    grand_action_totals = {
        action_type: voter_action_totals.get(action_type, 0)
        + candidate_action_totals.get(action_type, 0)
        for action_type in ["POST", "REPLY", "LIKE"]
    }

    print(f"\n### Grand Total Summary ###")
    print(f"Total Messages: {grand_total:,}")
    print("Breakdown by Action Type:")
    for action_type in ["POST", "REPLY", "LIKE"]:
        count = grand_action_totals.get(action_type, 0)
        percentage = (count / grand_total * 100) if grand_total > 0 else 0
        print(f"  {action_type}: {count:,} ({percentage:.1f}%)")

    print("\n" + "=" * 50 + "\n")

    create_combined_horizontal_stacked_bar_chart(
        voter_actions, candidate_actions, type_data
    )

In [40]:
plot_action_types_by_model(data, type_data)


### Voters Action Summary ###
Total Messages: 33,258
Breakdown by Action Type:
  POST: 3,314 (10.0%)
  REPLY: 16,220 (48.8%)
  LIKE: 13,724 (41.3%)

### Candidates Action Summary ###
Total Messages: 6,793
Breakdown by Action Type:
  POST: 403 (5.9%)
  REPLY: 3,891 (57.3%)
  LIKE: 2,499 (36.8%)

### Grand Total Summary ###
Total Messages: 40,051
Breakdown by Action Type:
  POST: 3,717 (9.3%)
  REPLY: 20,111 (50.2%)
  LIKE: 16,223 (40.5%)


